## Simple benchmark using LGB

Benchamrk with LGB

In [1]:
import os
import sys
from datetime import datetime

import gc

import eli5
import lightgbm as lgb
import numpy as np
import pandas as pd
import xgboost as xgb
from catboost import CatBoost, Pool
from IPython.display import display
from sklearn import metrics
from sklearn.metrics import make_scorer, roc_auc_score, mean_squared_error

In [2]:
sys.path.insert(0, "/opt/vssexclude/personal/kaggle/k_tab_aug")

In [3]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [4]:
import logging

import pandas as pd
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import TimeSeriesSplit

import src.common as common
import src.munging as process_data
import src.config.constants as constants
import src.cv as cv
import src.modeling as model

import os
from datetime import datetime
from timeit import default_timer as timer

from sklearn.model_selection import StratifiedKFold

import src.common as common
import src.config.constants as constants
import src.modeling.train_util as model
import src.munging.process_data_util as process_data

In [60]:
logger = common.get_logger('sub_1')
RUN_ID = datetime.now().strftime("%m%d_%H%M")
MODEL_NAME = "lgb_benhmark"
SEED = 42

train_df, test_df, sample_submission_df = process_data.read_processed_data(logger, 
                                                                           constants.PROCESSED_DATA_DIR, 
                                                                           train=True, 
                                                                           test=True, 
                                                                           sample_submission=True)

[INFO]2021-08-07 13:18:33,091:sub_1:Reading Data from /opt/vssexclude/personal/kaggle/k_tab_aug/data/processed...
[INFO]2021-08-07 13:18:33,091:sub_1:Reading Data from /opt/vssexclude/personal/kaggle/k_tab_aug/data/processed...
[INFO]2021-08-07 13:18:33,091:sub_1:Reading Data from /opt/vssexclude/personal/kaggle/k_tab_aug/data/processed...
[INFO]2021-08-07 13:18:33,214:sub_1:Shape of train_df : (250000, 101)
[INFO]2021-08-07 13:18:33,214:sub_1:Shape of train_df : (250000, 101)
[INFO]2021-08-07 13:18:33,214:sub_1:Shape of train_df : (250000, 101)
[INFO]2021-08-07 13:18:33,274:sub_1:Shape of test_df : (150000, 100)
[INFO]2021-08-07 13:18:33,274:sub_1:Shape of test_df : (150000, 100)
[INFO]2021-08-07 13:18:33,274:sub_1:Shape of test_df : (150000, 100)
[INFO]2021-08-07 13:18:33,284:sub_1:Shape of sample_submission_df : (150000, 1)
[INFO]2021-08-07 13:18:33,284:sub_1:Shape of sample_submission_df : (150000, 1)
[INFO]2021-08-07 13:18:33,284:sub_1:Shape of sample_submission_df : (150000, 1)


In [44]:
train_df.shape, test_df.shape, sample_submission_df.shape

((250000, 101), (150000, 100), (150000, 1))

In [ ]:
rare_classes = ["38", "39", "40"]

In [64]:
train_df_rare = train_df[train_df.loss == 42]

train_df = train_df.append([train_df_rare, train_df_rare, train_df_rare, train_df_rare], ignore_index=True)

In [66]:
temp.tail()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f91,f92,f93,f94,f95,f96,f97,f98,f99,loss
250007,0.834690,117,-2.117770,0.589045,4051.159912,18.96030,17.615299,-0.008741,-20.292000,1.14865,...,-60.402401,20.552601,1.77570,0.331692,8.40803,2.16598,0.481751,1.508780,11.59620,42
250008,0.502284,23,0.199544,-0.047501,1060.780029,14.00550,285.673004,0.841536,93.363297,1.25908,...,46.822800,22.452499,1.44078,-1.195910,-9.07898,2.06591,0.130087,1.383020,16.29130,42
250009,0.404423,29,-0.555755,-0.987523,126.855003,4.93123,717.114990,0.541411,12.024200,1.35534,...,-263.789001,48.836498,1.47618,-0.657360,-2.18131,2.68415,0.790616,0.800372,2.81641,42
250010,0.834690,117,-2.117770,0.589045,4051.159912,18.96030,17.615299,-0.008741,-20.292000,1.14865,...,-60.402401,20.552601,1.77570,0.331692,8.40803,2.16598,0.481751,1.508780,11.59620,42
250011,0.502284,23,0.199544,-0.047501,1060.780029,14.00550,285.673004,0.841536,93.363297,1.25908,...,46.822800,22.452499,1.44078,-1.195910,-9.07898,2.06591,0.130087,1.383020,16.29130,42


In [45]:
train_df.loss.nunique()

43

In [46]:
train_df.loss.value_counts()

0     60144
2     22225
1     22069
3     15902
4     13371
5     12458
6     10697
7      8894
8      8264
9      7435
10     7242
11     6572
12     5718
13     4828
14     4182
15     3671
16     3580
17     3420
18     3354
19     3079
20     2878
21     2478
22     2203
23     1875
24     1722
25     1457
26     1385
27     1284
28     1129
29     1096
30     1057
31      899
32      852
33      663
34      570
35      485
36      361
37      266
38      135
39       62
40       24
41       11
42        3
Name: loss, dtype: int64

In [9]:
LOGGER_NAME = "sub_1"
logger = logging.getLogger()
common.set_seed(SEED)

In [10]:
TARGET = "loss"

MODEL_TYPE = "lgb"
OBJECTIVE = "multiclass"
NUM_CLASSES = 41
METRIC = "multi_logloss"
BOOSTING_TYPE = "gbdt"
VERBOSE = 100
N_THREADS = -1
NUM_LEAVES = 31
MAX_DEPTH = -1
N_ESTIMATORS = 1000
LEARNING_RATE = 0.1
EARLY_STOPPING_ROUNDS = 100

lgb_params = {
    "objective": OBJECTIVE,
    "boosting_type": BOOSTING_TYPE,
    "learning_rate": LEARNING_RATE,
    "num_classes": NUM_CLASSES,
    "num_leaves": NUM_LEAVES,
    "tree_learner": "serial",
    "n_jobs": N_THREADS,
    "seed": SEED,
    "max_depth": MAX_DEPTH,
    "max_bin": 255,
    "metric": METRIC,
    "verbose": -1,
}

In [12]:
logger.info(train_df.columns)

train_X = train_df.drop([TARGET], axis=1)
train_Y = train_df[TARGET]
test_X = test_df

logger.info(
    f"Shape of train_X : {train_X.shape}, test_X: {test_X.shape}, train_Y: {train_Y.shape}"
)

predictors = list(train_X.columns)
sk = StratifiedKFold(n_splits=2, shuffle=True)

common.update_tracking(RUN_ID, "no_of_features", len(predictors), is_integer=True)
common.update_tracking(RUN_ID, "cv_method", "StratifiedKFold")

results_dict = model.lgb_train_validate_on_cv(
    logger=logger,
    run_id=RUN_ID,
    train_X=train_X,
    train_Y=train_Y,
    test_X=test_X,
    metric="log_loss",
    num_class=NUM_CLASSES,
    kf=sk,
    features=predictors,
    params=lgb_params,
    n_estimators=N_ESTIMATORS,
    early_stopping_rounds=EARLY_STOPPING_ROUNDS,
    cat_features="auto",
    is_test=False,
    verbose_eval=100,
)

Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 33.1776	valid_1's multi_logloss: 33.39
Early stopping, best iteration is:
[2]	training's multi_logloss: 2.33433	valid_1's multi_logloss: 2.99164
Training until validation scores don't improve for 100 rounds
[100]	training's multi_logloss: 28.0966	valid_1's multi_logloss: 28.0786
Early stopping, best iteration is:
[2]	training's multi_logloss: 2.33383	valid_1's multi_logloss: 3.00334


In [41]:
results_dict_copy = results_dict.copy()

results_dict_copy["prediction"] = np.argmax(results_dict["prediction"], axis=1)

results_dict_copy["y_oof"] = np.argmax(results_dict["y_oof"], axis=1)

In [42]:
results_dict_copy

{'y_oof': array([0, 0, 0, ..., 0, 0, 0]),
 'prediction': array([[2.12596238e-01, 7.76788345e-02, 1.05202873e-01, ...,
         4.28089813e-04, 2.43989520e-04, 1.81865916e-04],
        [2.20404233e-01, 1.09968831e-01, 8.64317438e-02, ...,
         4.40048657e-04, 2.52451708e-04, 1.91062326e-04],
        [2.30517050e-01, 8.51830706e-02, 8.52294557e-02, ...,
         4.68692679e-04, 2.67852008e-04, 2.00917565e-04],
        ...,
        [2.28254419e-01, 7.86362279e-02, 1.12195487e-01, ...,
         4.32314311e-04, 2.47662704e-04, 1.86824522e-04],
        [2.16248405e-01, 7.50646609e-02, 8.17812846e-02, ...,
         4.26724767e-04, 2.43410597e-04, 1.81783521e-04],
        [2.18234128e-01, 9.37509166e-02, 7.94200418e-02, ...,
         4.22817383e-04, 2.44548058e-04, 1.88535506e-04]]),
 'oof_score': 2.99749,
 'cv_scores': [2.9916398112184024, 3.0033389524308967],
 'avg_cv_scores': 2.99749,
 'std_cv_scores': 0.00585,
 'feature_importance':    feature  importance  fold
 0       f0          70 

In [13]:
results_dict.keys()

dict_keys(['y_oof', 'prediction', 'oof_score', 'cv_scores', 'avg_cv_scores', 'std_cv_scores', 'feature_importance', 'best_features'])

In [30]:
list(np.argmax(results_dict["y_oof"], axis=1))

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 38,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 22,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 38,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 38,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 24,
 0,
 0,
 0,
 0,
 0,
 0,
 36,
 0,
 0,
 0,
 0,
 0,
 12,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 19,
 0,
 32,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 38,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 29,
 0,
 0,
 0,
 0,
 1,
 0,
 36,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [26]:
list(np.argmax(results_dict["prediction"], axis=1))

[0,
 0,
 0,
 0,
 38,
 0,
 38,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 35,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 38,
 0,
 0,
 0,
 0,
 0,
 0,
 36,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 38,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 38,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 38,
 0,
 0,
 38,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 38,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 36,
 0,
 0,
 0,
 0,
 0,
 